### Using Spotipy wrapper to gather data from the spotify API

In [3]:
# set clientID and client secret in global variables
import getpass 
clientid = '297ada524bde40a8a7e580b18966dbba'
clientsecret = getpass.getpass() 

········


In [4]:
#import the wrapper
import spotipy
#import the login part 
from spotipy.oauth2 import SpotifyClientCredentials
# import pandas
import pandas as pd

In [5]:
#connect to the api
sp=spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=clientid, client_secret=clientsecret))

In [6]:
# test with a simple search if we get our results

results = sp.search(q='august burns red', limit=20) # store search query in a variable
for trackid, name in enumerate(results['tracks']['items']): # iterate over "track" & "items" keys
    print(trackid, name['name']) # extract the id and the name

0 Chop Suey!
1 Poor Millionaire
2 White Washed
3 Paramount
4 Mariana's Trench
5 Defender
6 Icarus
7 Invisible Enemy
8 The Legend Of Zelda
9 Composure
10 Paramount - Reprise
11 Standing In The Storm
12 Ghosts
13 August Burns Red
14 Westworld
15 Wrecking Ball
16 King Of Sorrow
17 Bones
18 The Frost
19 Meddler


In [7]:
# fetching some playlists

playlist_ids = [] # empty list for playlist
playlists = sp.user_playlists('spotify') # I am using playlists created by the spotify user to not introduce bias

while playlists: # while playlist = true
    for playlist in (playlists['items']): #iterate over the spotify_playlists_items
        playlist_ids.append(playlist['id']) # append the uri to the playlist list
    if playlists['next']: # check if there is a next playlist available
        playlists = sp.next(playlists)
    else:
        playlists = None # exit condition

In [8]:
len(set(playlist_ids))        

1398

In [9]:
len(playlist_ids)

1398

## I now fetched 1398 unique playlists created by the spotify user
#### next step: Getting the songs from the playlists into a dataframe

In [36]:
# define a function to get tracks from paylists

def get_songname(playlist_id):
    result = sp.user_playlist_tracks("spotify",playlist_id) # call the playlist directly and store it in result
    names = result['items'] #navigate to the item key that contain the songnames
    while result['next']: # while a next track is avaible
        result = sp.next(result) # call next song
        names.extend(result['items']) # extend name from dictionary
    return names

In [37]:
# Trying the function with one playlist

In [38]:
playlist_ids[0]

'37i9dQZF1DXcBWIGoYBM5M'

In [39]:
tracks = get_songname("37i9dQZF1DXcBWIGoYBM5M")
tracks

[{'added_at': '2021-05-03T15:49:35Z',
  'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/'},
   'href': 'https://api.spotify.com/v1/users/',
   'id': '',
   'type': 'user',
   'uri': 'spotify:user:'},
  'is_local': False,
  'primary_color': None,
  'track': {'album': {'album_type': 'single',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6qqNVTkY8uBg9cP3Jd7DAH'},
      'href': 'https://api.spotify.com/v1/artists/6qqNVTkY8uBg9cP3Jd7DAH',
      'id': '6qqNVTkY8uBg9cP3Jd7DAH',
      'name': 'Billie Eilish',
      'type': 'artist',
      'uri': 'spotify:artist:6qqNVTkY8uBg9cP3Jd7DAH'}],
    'available_markets': ['AD',
     'AE',
     'AG',
     'AL',
     'AM',
     'AO',
     'AR',
     'AT',
     'AU',
     'AZ',
     'BA',
     'BB',
     'BD',
     'BE',
     'BF',
     'BG',
     'BH',
     'BI',
     'BJ',
     'BN',
     'BO',
     'BR',
     'BS',
     'BT',
     'BW',
     'BY',
     'BZ',
     'CA',
     'CH',
     'CI',
   

In [ ]:
df = pd.DataFrame(tracks) # store the result in a pandas dataframe for easier handling 
df

In [ ]:
tracks_playlist = df['track'] # singel out for the tracks columns into a new dataframe
tracks_playlist

In [50]:
# now find the artist in that dictionary
tracks_playlist[0]['album']['artists'][0]['name']


'Billie Eilish'

In [51]:
# find the songname
tracks_playlist[0]['name']


'Your Power'

In [52]:
# find the song id
tracks_playlist[0]['id']


'5qNh5WtzMbfpSj2jLlBkoD'

In [53]:
# get the audio features for that track with a new request
sp.audio_features('47EiUVwUp4C9fGccaPuUCS')[0]


{'danceability': 0.731,
 'energy': 0.573,
 'key': 4,
 'loudness': -10.059,
 'mode': 0,
 'speechiness': 0.0544,
 'acousticness': 0.401,
 'instrumentalness': 5.22e-05,
 'liveness': 0.113,
 'valence': 0.145,
 'tempo': 109.928,
 'type': 'audio_features',
 'id': '47EiUVwUp4C9fGccaPuUCS',
 'uri': 'spotify:track:47EiUVwUp4C9fGccaPuUCS',
 'track_href': 'https://api.spotify.com/v1/tracks/47EiUVwUp4C9fGccaPuUCS',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/47EiUVwUp4C9fGccaPuUCS',
 'duration_ms': 205090,
 'time_signature': 4}

In [93]:
# define a function to convert playlists into dataframes
def playlist_to_df(playlist_id):
    data = get_songname(playlist_id) # calling the function that gets songnames
    datadf = pd.DataFrame(data) # convert to dataframe
    songs =datadf['track'] # extract track column
    out = scrape_playlist_data(songs)
    return out

We got everything we need from the song.
Now scaling it up 

In [90]:
def scrape_playlist_data(playlist_name):
    
    df = pd.DataFrame(columns=['artist','title','tid','danceability','energy','key',
                               'loudness','mode','speechiness','acousticness','instrumentalness',
                               'liveness','valence','tempo'])
    
    for i in range(len(playlist_name)): # iterate over the playlist
        if len(playlist_name[i]['album']['artists']) == 1: #check if the song only has one artist
            artist = playlist_name[i]['album']['artists'][0]['name'] # extract artist name
            artist = artist.lower()
        else:
            artist = playlist_name[i]['album']['artists'][0]['name'] + ", " + playlist_name[i]['album']['artists'][1]['name'] # if we have two artist, extract both
            artist = artist.lower() # lowercase
        title = playlist_name[i]['name'] # get name if the song
        title=title.lower()# lowercase
        tid = playlist_name[i]['id']
        
        features = sp.audio_features(tid) # now call the features of the song using the extracted id
        
        features = features[0]
        danceability = features['danceability']
        energy = features['energy']
        key = features['key']
        loudness = features['loudness']
        mode = features['mode']
        speechiness = features['speechiness']
        acousticness = features['acousticness']
        instrumentalness = features['instrumentalness']
        liveness = features['liveness']
        valence = features['valence']
        tempo = features['tempo']
        featurelist = [artist, title, tid, danceability, energy, key, loudness, mode, speechiness, acousticness, instrumentalness, liveness, valence, tempo]
        feature_series = pd.Series(featurelist, index = df.columns)
        df = df.append(feature_series, ignore_index=True)
    return df

In [91]:
songdatabase = scrape_playlist_data(tracks_playlist)


In [92]:
songdatabase.tail()

,artist,title,tid,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
45,olivia rodrigo,drivers license,7lPN2DXiMsVn7XUKtOW1CS,0.585,0.436,10,-8.761,1,0.0601,0.721000,0.000013,0.105,0.132,143.874
46,rosé,on the ground,2pn8dNVSpYnAtlKFC8Q0DJ,0.311,0.607,1,-6.578,1,0.1100,0.017400,0.000000,0.131,0.286,188.700
47,twenty one pilots,shy away,2z0IupRlVRlDN5r2IVqHyN,0.588,0.856,9,-5.242,0,0.0316,0.000818,0.073900,0.238,0.635,96.966
48,"lil tjay, 6lack",calling my phone,5Kskr9LcNYa0tpt5f0ZEJx,0.907,0.393,4,-7.636,0,0.0539,0.451000,0.000001,0.135,0.202,104.949
49,"bad bunny, jhay cortez",dákiti,47EiUVwUp4C9fGccaPuUCS,0.731,0.573,4,-10.059,0,0.0544,0.401000,0.000052,0.113,0.145,109.928


In [ ]:
def playlist_to_df(playlist_id):
    data = get_songname(playlist_id) # calling the function that gets details
    datadf = pd.DataFrame(data) # convert to dataframe
    songs =datadf['track'] # extract track column
    out = scrape_playlist_data(songs)
    return out

In [95]:
test = playlist_to_df('37i9dQZF1DXcBWIGoYBM5M')
test

,artist,title,tid,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,billie eilish,your power,5qNh5WtzMbfpSj2jLlBkoD,0.634,0.285,9,-14.007,0,0.0807,0.937000,0.000359,0.2320,0.203,129.650
1,"doja cat, sza",kiss me more (feat. sza),748mdHapucXQri7IAO8yFK,0.762,0.701,8,-3.541,1,0.0286,0.235000,0.000158,0.1230,0.742,110.968
2,"dua lipa, dababy",levitating (feat. dababy),463CkQjx2Zk1yXoBuierM9,0.702,0.825,6,-3.787,0,0.0601,0.008830,0.000000,0.0674,0.915,102.977
3,lil nas x,montero (call me by your name),3BZEcbdtXQSo7OrvKRJ6mb,0.610,0.508,8,-6.682,0,0.1520,0.297000,0.000000,0.3840,0.758,178.818
4,"the weeknd, ariana grande",save your tears (with ariana grande) (remix),37BZB0z9T8Xu7U3e65qxFy,0.650,0.825,0,-4.645,1,0.0325,0.021500,0.000024,0.0936,0.593,118.091
5,justin bieber,peaches (feat. daniel caesar & giveon),4iJyoBOLtHqaGxP12qzhQI,0.677,0.696,0,-6.181,1,0.1190,0.321000,0.000000,0.4200,0.464,90.030
6,"the kid laroi, miley cyrus",without you (with miley cyrus),1EHUQesfVKwbthtNcpBi9g,0.655,0.469,0,-6.257,1,0.0292,0.173000,0.000000,0.0956,0.467,92.560
7,polo g,rapstar,43PGPuHIlVOc04jrZVh9L6,0.789,0.536,6,-6.862,1,0.2420,0.410000,0.000000,0.1290,0.437,81.039
8,masked wolf,astronaut in the ocean,3Ofmpyhv5UAQ70mENzB277,0.778,0.695,4,-6.865,0,0.0913,0.175000,0.000000,0.1500,0.472,149.996
9,"travis scott, hvme",goosebumps - remix,5uEYRdEIh9Bo4fpjDd4Na9,0.841,0.593,1,-7.846,1,0.0379,0.418000,0.000000,0.1240,0.808,124.917


In [96]:
playlist_ids

['37i9dQZF1DXcBWIGoYBM5M',
 '37i9dQZF1DX0XUsuxWHRQd',
 '37i9dQZF1DX1lVhptIYRda',
 '37i9dQZF1DX10zKzsJ2jva',
 '37i9dQZF1DX4JAvHpjipBk',
 '37i9dQZF1DX4sWSpwq3LiO',
 '37i9dQZF1DX4SBhb3fqCJd',
 '37i9dQZF1DWXRqgorJj26U',
 '37i9dQZF1DX4dyzvuaRJ0n',
 '37i9dQZF1DXcF6B6QPhFDv',
 '37i9dQZF1DWXJfnUiYjUKT',
 '37i9dQZF1DXcRXFNfZr7Tp',
 '37i9dQZF1DX4o1oenSJRJd',
 '37i9dQZF1DXbTxeAdrVG2l',
 '37i9dQZF1DX4UtSsGT1Sbe',
 '37i9dQZF1DWTJ7xPn4vNaz',
 '37i9dQZF1DXaKIA8E7WcJj',
 '37i9dQZF1DWSV3Tk4GO2fq',
 '37i9dQZF1DWTwnEm1IYyoj',
 '37i9dQZF1DX2A29LI7xHn1',
 '37i9dQZF1DX2RxBh64BHjQ',
 '37i9dQZF1DWVA1Gq4XHa6U',
 '37i9dQZF1DWY4xHQp97fN6',
 '37i9dQZF1DWX3387IZmjNa',
 '37i9dQZF1DWYkaDif7Ztbp',
 '37i9dQZF1DX5hR0J49CmXC',
 '37i9dQZF1DXan38dNVDdl4',
 '37i9dQZF1DWSvKsRPPnv5o',
 '37i9dQZF1DWUVpAXiEPK8P',
 '37i9dQZF1DX0Tkc6ltcBfU',
 '37i9dQZF1DX1YPTAhwehsC',
 '37i9dQZF1DWTggY0yqBxES',
 '37i9dQZF1DX0HRj9P7NxeE',
 '37i9dQZF1DWT6SJaitNDax',
 '37i9dQZF1DX2r0FByV5U4C',
 '37i9dQZF1DXaKctwWdt4be',
 '37i9dQZF1DWT2jS7NwYPVI',
 

In [101]:
database = pd.DataFrame(columns=['artist','title','tid','danceability','energy','key',
                               'loudness','mode','speechiness','acousticness','instrumentalness',
                               'liveness','valence','tempo'])


,artist,title,tid,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,various artists,nicholas sparks introduction,47ugCLTk9gxAjV0gMcuVjv,0,0.7770,11,-7.597,1,0,0.25000,0,0.4160,0,0
1,jd eicher,two by two,0AlpTXa6gsE8RnqKv8sofP,0.461,0.6220,0,-5.531,1,0.0381,0.59900,0,0.0815,0.445,152.079
2,various artists,sparks reaches out to jd,0ovvdxdXgA1liaoO6HqzqH,0.689,0.5930,8,-7.577,1,0.849,0.60500,0,0.3350,0.867,62.823
3,jd eicher,love is gonna find you,60E1rDrHzNuFrXoIcwkDRL,0.688,0.3130,6,-11.971,1,0.039,0.80000,0,0.1440,0.436,127.014
4,various artists,the creative process,2DD2pHlM7YhSf63FkjZNjC,0.727,0.5960,10,-8.013,1,0.895,0.68700,0,0.5880,0.675,67.962
5,jd eicher,the last love song,00H0OITPdGkJvDERkOb11s,0.49,0.2530,9,-9.750,1,0.0357,0.78600,0,0.0926,0.306,134.667
6,various artists,the initial meeting,7qQ77NXf54uOmjkGOgXjpY,0.617,0.6540,10,-8.404,1,0.691,0.87900,0,0.3700,0.798,91.254
7,jd eicher,not afraid,3o6L6Bo8IlYRZQDnChQbiO,0.627,0.5600,0,-7.359,1,0.0335,0.26400,0,0.0838,0.311,73.501
8,various artists,how long does writing a song take?,3iynFJXBY7nkstKQ6M30oa,0.784,0.6820,8,-7.926,1,0.673,0.90900,0,0.3980,0.829,89.325
9,jd eicher,the last love song,2UYyRfpX1KZzkCenyejmPJ,0.528,0.8020,10,-7.966,1,0.0377,0.27000,0,0.0948,0.311,147.001


In [109]:
temp = playlist_to_df('37i9dQZF1DWUFmyho2wkQU')
database = database.append(temp)


In [110]:
database.tail()

,artist,title,tid,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
145,trick daddy,nann nigga,7d9I42jF759n5HuUeoulzR,0.837,0.819,2,-5.245,1,0.211,0.000507,0,0.3230,0.645,149.918
146,outkast,the way you move (feat. sleepy brown) - radio mix,1czXAuXNIGMhEkeSl08C5W,0.869,0.608,5,-4.866,0,0.0474,0.137000,0.000087,0.0624,0.627,125.992
147,lloyd banks,i'm so fly,0MjM12n2Blr3UxS083X5cQ,0.629,0.625,8,-4.258,0,0.317,0.146000,0,0.0756,0.789,179.666
148,twista,slow jamz (feat. kanye west & jamie foxx),1boHm9rjmvzxpJASd9dnGK,0.561,0.764,2,-6.645,0,0.138,0.373000,0,0.1570,0.598,145.158
149,ludacris,what's your fantasy (featuring shawna),4LwOrnuxJwR7C5Sw4liY4Z,0.915,0.790,6,-3.186,0,0.207,0.160000,0,0.1120,0.967,140.095


In [121]:
playlistsids2 = pd.read_csv('/Users/ohmat/Desktop/607ids.csv')
playlistsids2

,Unnamed: 0,playlist
0,0,37i9dQZF1DWUFmyho2wkQU
1,1,37i9dQZF1DZ06evO1Xa1Op
2,2,37i9dQZF1DXcbGhAooTSJy
3,3,37i9dQZF1DWZMWLrh2UzwC
4,4,37i9dQZF1DX6uhsAfngvaD
...,...,...
602,602,37i9dQZF1DX10uaP8FoDNe
603,603,37i9dQZF1DX49poIUZYXp7
604,604,37i9dQZF1DX7Qo2zphj7u3
605,605,37i9dQZF1DWSkkUxEhrBdF


In [123]:
finalids = playlistsids2['playlist'].tolist()
finalids

['37i9dQZF1DWUFmyho2wkQU',
 '37i9dQZF1DZ06evO1Xa1Op',
 '37i9dQZF1DXcbGhAooTSJy',
 '37i9dQZF1DWZMWLrh2UzwC',
 '37i9dQZF1DX6uhsAfngvaD',
 '37i9dQZF1DWYmmr74INQlb',
 '37i9dQZF1DWZqUHC2tviPw',
 '37i9dQZF1DXbgLhPxSBEKX',
 '37i9dQZF1DWYRTlrhMB12D',
 '37i9dQZF1DWTJ7xPn4vNaz',
 '37i9dQZF1DX5Ejj0EkURtP',
 '37i9dQZF1DX50QitC6Oqtn',
 '37i9dQZF1DWXbttAJcbphz',
 '37i9dQZF1DX8SE5tIsUWTD',
 '37i9dQZF1DX7HGyCQ2dcNx',
 '37i9dQZF1DX4qKWGR9z0LI',
 '37i9dQZF1DX0z1epx5KTtS',
 '37i9dQZF1DX9udeSemCofA',
 '37i9dQZF1DWU0ScTcjJBdj',
 '37i9dQZF1DX2E5psOxzjXz',
 '37i9dQZF1DX4WgZiuR77Ef',
 '37i9dQZF1DX2zAr9vdmFlU',
 '37i9dQZF1DWXnscMH24yOc',
 '37i9dQZF1DX8RXxAW8DgJV',
 '37i9dQZF1DWUptwB9dhZvQ',
 '37i9dQZF1DXdx7sCF75xKy',
 '37i9dQZF1DWWnpcjfCqaW0',
 '37i9dQZF1DXbn7xTVabEKT',
 '37i9dQZF1DX8FwnYE6PRvL',
 '37i9dQZF1DWYTn3LtYLd3y',
 '37i9dQZF1DWZhxU4AiByxO',
 '37i9dQZF1DWZeKCadgRdKQ',
 '37i9dQZF1DWWi0hHcPHnic',
 '37i9dQZF1DWYAyGN1XM1YS',
 '37i9dQZF1DWW7gj0FcGEx6',
 '37i9dQZF1DXc4SqyiYXF2n',
 '37i9dQZF1DX5KJJSIYaNvs',
 

In [128]:
database = pd.DataFrame(columns=['artist','title','tid','danceability','energy','key',
                               'loudness','mode','speechiness','acousticness','instrumentalness',
                               'liveness','valence','tempo'])

In [134]:
for playlist in finalids[300:]:
    temp = playlist_to_df(playlist)
    database = database.append(temp)

TypeError: 'NoneType' object is not subscriptable

In [135]:
database

,artist,title,tid,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,waka flocka flame,no hands (feat. roscoe dash & wale),03tqyYWC9Um2ZqU0ZN849H,0.760,0.595,1,-6.366,1,0.0391,0.005440,0,0.2410,0.361,131.497
1,trick daddy,take it to da house,1wLhwzPAtze5JT8qqXlqaO,0.935,0.780,6,-5.795,0,0.1970,0.015800,0,0.1640,0.918,123.895
2,dmx,ruff ryders' anthem,1BKT2I9x4RGKaKqW4up34s,0.880,0.577,10,-8.093,0,0.3020,0.056400,0.000001,0.2010,0.740,90.991
3,ugk,int'l players anthem (i choose you) (feat. out...,4ns1XFP3W5JPyzvnAjMdHD,0.675,0.680,4,-5.010,0,0.2560,0.065500,0,0.3540,0.845,155.698
4,lil jon & the east side boyz,what u gon' do,1bTphHXspys3RvvRxfzzIc,0.838,0.724,1,-6.900,0,0.2420,0.000077,0.000001,0.1620,0.537,150.192
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,orville peck,dead of night,4WDazmiEd5PyY3taS11VfD,0.454,0.548,2,-11.618,1,0.0547,0.820000,0.356,0.0606,0.500,99.892
46,michaela anne,desert dove,7oC1Lo7hqhpxIU8d5iH6cc,0.592,0.419,2,-9.441,1,0.0296,0.349000,0.00102,0.0896,0.370,119.868
47,joan shelley,coming down for you,4bDALATwj7AYLJ9h8cImWf,0.528,0.338,5,-10.942,0,0.0302,0.969000,0.0779,0.1600,0.555,92.262
48,che apalache,the dreamer,3TwNSH4IkiBSrFH0wwfmn1,0.295,0.426,3,-8.734,1,0.0343,0.824000,0.000157,0.1130,0.381,184.125


In [136]:
database_unique = database.drop_duplicates()
database_unique

,artist,title,tid,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,waka flocka flame,no hands (feat. roscoe dash & wale),03tqyYWC9Um2ZqU0ZN849H,0.760,0.595,1,-6.366,1,0.0391,0.005440,0,0.2410,0.361,131.497
1,trick daddy,take it to da house,1wLhwzPAtze5JT8qqXlqaO,0.935,0.780,6,-5.795,0,0.1970,0.015800,0,0.1640,0.918,123.895
2,dmx,ruff ryders' anthem,1BKT2I9x4RGKaKqW4up34s,0.880,0.577,10,-8.093,0,0.3020,0.056400,0.000001,0.2010,0.740,90.991
3,ugk,int'l players anthem (i choose you) (feat. out...,4ns1XFP3W5JPyzvnAjMdHD,0.675,0.680,4,-5.010,0,0.2560,0.065500,0,0.3540,0.845,155.698
4,lil jon & the east side boyz,what u gon' do,1bTphHXspys3RvvRxfzzIc,0.838,0.724,1,-6.900,0,0.2420,0.000077,0.000001,0.1620,0.537,150.192
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44,wilco,love is everywhere (beware),7sxr5Pvovng4ECTiCqeSWm,0.313,0.520,7,-9.436,1,0.0298,0.120000,0.452,0.0993,0.511,144.249
45,orville peck,dead of night,4WDazmiEd5PyY3taS11VfD,0.454,0.548,2,-11.618,1,0.0547,0.820000,0.356,0.0606,0.500,99.892
46,michaela anne,desert dove,7oC1Lo7hqhpxIU8d5iH6cc,0.592,0.419,2,-9.441,1,0.0296,0.349000,0.00102,0.0896,0.370,119.868
47,joan shelley,coming down for you,4bDALATwj7AYLJ9h8cImWf,0.528,0.338,5,-10.942,0,0.0302,0.969000,0.0779,0.1600,0.555,92.262


In [138]:
database_unique.to_csv('spotifydata.csv')